# Segmenting and Clustering Neighborhoods in Ahmedabad and Predicting Housing Sales Price


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json # library to handle JSON files

from geopy.geocoders import Nominatim
from googlemaps import Client as googlemaps# convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Getting Data Ready

In [2]:
df = pd.read_csv('/Users/ishaanbuch/Desktop/AhmedabadPostalCodes.csv',header=None)
df.columns=["Location","Name","Pincode","State","District"]
#indexNames = df[ df['Borough'] =='Not assigned'].index
#df.drop(indexNames , inplace=True)
#df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
#result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
#df_new=result.reset_index()
df['Lat']=""
df['Long']=""
df.head()

,Location,Name,Pincode,State,District,Lat,Long
0,Abasana,Abasana,382120,Gujarat,Ahmedabad,,
1,Adroda,Adroda,382220,Gujarat,Ahmedabad,,
2,Adval,Adval,382460,Gujarat,Ahmedabad,,
3,Ahmedabad.,Ahmedabad.,380001,Gujarat,Ahmedabad,,
4,Alampur,Alampur,363610,Gujarat,Ahmedabad,,


In [3]:
abadprices = pd.read_csv("/Users/ishaanbuch/Desktop/AhmedabadHousingPrices.csv")
#print(abadprices["Trends"][234])
country_geo='/Users/ishaanbuch/Desktop/Wards.geojson'
abadprices.replace('-',np.nan)
abadprices.dropna(how='any', inplace=True)

In [4]:
abadprices

,Location,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends
0,Bopal,393,"12,500 / sqft","8,15.91 / sqft",24.5%
1,Chandkheda,"2,047","8,302 / sqft","6,632.57 / sqft",21.9%
2,"Satellite, Ahmedabad","3,678","12,222 / sqft","9,394.24 / sqft",56.9%
3,Nirma University,"1,927","8,032 / sqft","6,361.6 / sqft",-40.2%
4,Motera,"2,597","4,762 / sqft","4,608.2 / sqft",33.2%
5,Thaltej,"3,472","15,371 / sqft","11,756.57 / sqft",33.6%
6,Prahlad Nagar,"2,381","13,793 / sqft","11,256.8 / sqft",20.9%
7,"Shela, Ahmedabad","1,950","27,347 / sqft","7,515.48 / sqft",22.3%
8,Vaishno Devi Circle,"2,755","8,034 / sqft","7,234.79 / sqft",8.1%
9,Bodakdev,"2,653","22,500 / sqft","11,139.85 / sqft",-


In [5]:
joined=pd.merge(df,abadprices,on=['Location'], how='left')
df=joined
df

,Location,Name,Pincode,State,District,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends
0,Abasana,Abasana,382120,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
1,Adroda,Adroda,382220,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
2,Adval,Adval,382460,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
3,Ahmedabad.,Ahmedabad.,380001,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
4,Alampur,Alampur,363610,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
5,Ambaliara,Ambaliara,387810,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
6,Ambareli,Ambareli,387810,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
7,Ambawadi,Ambawadi,380006,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
8,Ambawadi Vistar,Ambawadi Vistar,380015,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN
9,Ambli,Ambli,382463,Gujarat,Ahmedabad,,,NaN,NaN,NaN,NaN


In [6]:
for x in range(0,len(df),1):
    try:
        geolocator = Nominatim(user_agent="Ahmedabad")
        geocode_result = geolocator.geocode(df['Pincode'][x])
        df['Lat'][x] = geocode_result.latitude
        df['Long'][x] = geocode_result.longitude
        #areas['Pincode'][x]=geocode_result.pincode
        #print(geocode_result)
        #print(geocode_result[0])
        #print(x)
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occurred.", e )
        print(x)

/Users/ishaanbuch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/ishaanbuch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.head()

,Location,Name,Pincode,State,District,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends
0,Abasana,Abasana,382120,Gujarat,Ahmedabad,23.3662,72.054,NaN,NaN,NaN,NaN
1,Adroda,Adroda,382220,Gujarat,Ahmedabad,22.8579,72.4062,NaN,NaN,NaN,NaN
2,Adval,Adval,382460,Gujarat,Ahmedabad,22.3755,71.9772,NaN,NaN,NaN,NaN
3,Ahmedabad.,Ahmedabad.,380001,Gujarat,Ahmedabad,23.0198,72.5935,NaN,NaN,NaN,NaN
4,Alampur,Alampur,363610,Gujarat,Ahmedabad,22.3515,71.7118,NaN,NaN,NaN,NaN


In [8]:
locator = Nominatim(user_agent="Ahmedabad")
location = locator.geocode("Ahmedabad")
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
latitude_ahmedabad=location.latitude
longitude_ahmedabad=location.longitude

Latitude = 23.0216238, Longitude = 72.5797068


In [9]:
map_ahmedabad = folium.Map(location=[latitude_ahmedabad, longitude_ahmedabad], zoom_start=12)

# add markers to map
for lat, lng, Neighbourhood in zip(df['Lat'], df['Long'], df['Location']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ahmedabad)
    
map_ahmedabad.choropleth(geo_data=country_geo,
            data=abadprices,
             columns=['Location', 'Buy Rates 1'],
             #key_on='feature.id',
             fill_color='BuGn' ,fill_opacity=0.3, line_opacity=0.7,
             )
    
map_ahmedabad

/Users/ishaanbuch/opt/anaconda3/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'UJ4DSGYTRDPKM4AM0MNLL0ZZSYGR4AENLAFMM4JZHLGX3DHK' # your Foursquare ID
CLIENT_SECRET = 'OACRGHH5SAVTRLIA2OY50XBFZ2K4K0G5QOXGPIMEHJQZRSGU' # your Foursquare Secret
VERSION = '20191225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UJ4DSGYTRDPKM4AM0MNLL0ZZSYGR4AENLAFMM4JZHLGX3DHK
CLIENT_SECRET:OACRGHH5SAVTRLIA2OY50XBFZ2K4K0G5QOXGPIMEHJQZRSGU


In [11]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [12]:
import requests

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
ahmedabad_venues = getNearbyVenues(names=df['Location'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Long']
                                  )

Abasana


In [15]:
ahmedabad_venues.head(10)

,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ahmedabad.,23.019845,72.593461,Jay Shankar Sundri Hall,23.020832,72.592054,Theater
1,Ahmedabad.,23.019845,72.593461,Astodia Bhajiya House,23.017076,72.591230,Snack Place
2,Ahmedabad.,23.019845,72.593461,Astodia Darwaza,23.017156,72.591002,Historic Site
3,Ahmedabad.,23.019845,72.593461,Mangaldas ni Haveli II,23.023121,72.591255,Bed & Breakfast
4,Ahmedabad.,23.019845,72.593461,Lucky Tea,23.016723,72.590728,Tea Room
5,Ahmedabad.,23.019845,72.593461,Bhagyoday Restaurant,23.015734,72.592683,Indian Restaurant
6,Ambaliara,22.722057,72.436105,YES Bank ATM,22.719334,72.438113,Bank
7,Ambareli,22.722057,72.436105,YES Bank ATM,22.719334,72.438113,Bank
8,Ambawadi,23.023169,72.569912,The Westend,23.022107,72.567869,Hotel
9,Ambawadi,23.023169,72.569912,Wow Mughlai Handi and BBQ Grill,23.020332,72.569546,Indian Restaurant


In [16]:
ahmedabad_venues.shape

(742, 7)

In [17]:
ahmedabad_venues.groupby('Location').count()

,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Location,,,,,,
Ahmedabad.,6,6,6,6,6,6
Ambaliara,1,1,1,1,1,1
Ambareli,1,1,1,1,1,1
Ambawadi,11,11,11,11,11,11
Ambawadi Vistar,14,14,14,14,14,14
Amraiwadi,3,3,3,3,3,3
Anandnagar,2,2,2,2,2,2
Andej,1,1,1,1,1,1
Asarwa Chakla,5,5,5,5,5,5


## Analysing Each Neighborhood

In [18]:
# one hot encoding
ahmedabad_onehot = pd.get_dummies(ahmedabad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ahmedabad_onehot['Location'] = ahmedabad_venues['Location'] 

# move neighborhood column to the first column
fixed_columns = [ahmedabad_onehot.columns[-1]] + list(ahmedabad_onehot.columns[:-1])
ahmedabad_onehot.head()

,ATM,American Restaurant,Asian Restaurant,Auto Dealership,BBQ Joint,Bakery,Bank,Bed & Breakfast,Big Box Store,Bistro,Bookstore,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Clothing Store,Coffee Shop,Convenience Store,Cretan Restaurant,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court,Food Truck,Gift Shop,Golf Course,Greek Restaurant,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Industrial Estate,Internet Cafe,Italian Restaurant,Korean Restaurant,Lake,Lingerie Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Nightclub,Outdoor Supply Store,Park,Pharmacy,Pizza Place,Planetarium,Platform,Recreation Center,Restaurant,River,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Snack Place,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Club,Street Food Gathering,Supermarket,Tailor Shop,Taverna,Tea Room,Tennis Stadium,Theater,Travel Lounge,Tsipouro Restaurant,Vegetarian / Vegan Restaurant,Zoo,Location
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Ahmedabad.
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahmedabad.
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahmedabad.
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahmedabad.
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Ahmedabad.


In [19]:
ahmedabad_onehot.shape

(742, 91)

In [20]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
ahmedabad_grouped = ahmedabad_onehot.groupby('Location').mean().reset_index()
ahmedabad_grouped

,Location,ATM,American Restaurant,Asian Restaurant,Auto Dealership,BBQ Joint,Bakery,Bank,Bed & Breakfast,Big Box Store,Bistro,Bookstore,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Clothing Store,Coffee Shop,Convenience Store,Cretan Restaurant,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court,Food Truck,Gift Shop,Golf Course,Greek Restaurant,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Industrial Estate,Internet Cafe,Italian Restaurant,Korean Restaurant,Lake,Lingerie Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Nightclub,Outdoor Supply Store,Park,Pharmacy,Pizza Place,Planetarium,Platform,Recreation Center,Restaurant,River,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Snack Place,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Club,Street Food Gathering,Supermarket,Tailor Shop,Taverna,Tea Room,Tennis Stadium,Theater,Travel Lounge,Tsipouro Restaurant,Vegetarian / Vegan Restaurant,Zoo
0,Ahmedabad.,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.166667,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.166667,0.0,0.000000,0.000000,0.00
1,Ambaliara,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00
2,Ambareli,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00
3,Ambawadi,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.363636,0.000000,0.000000,0.000000,0.454545,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.090909,0.

#### Printing each neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in ahmedabad_grouped['Location']:
    print("----"+hood+"----")
    temp = ahmedabad_grouped[ahmedabad_grouped['Location'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ahmedabad.----
               venue  freq
0      Historic Site  0.17
1  Indian Restaurant  0.17
2            Theater  0.17
3    Bed & Breakfast  0.17
4           Tea Room  0.17


----Ambaliara----
           venue  freq
0           Bank   1.0
1            ATM   0.0
2  Moving Target   0.0
3       Platform   0.0
4    Planetarium   0.0


----Ambareli----
           venue  freq
0           Bank   1.0
1            ATM   0.0
2  Moving Target   0.0
3       Platform   0.0
4    Planetarium   0.0


----Ambawadi----
               venue  freq
0  Indian Restaurant  0.45
1              Hotel  0.36
2         Restaurant  0.09
3              River  0.09
4                ATM  0.00


----Ambawadi Vistar----
                  venue  freq
0             BBQ Joint  0.14
1     Indian Restaurant  0.14
2  Fast Food Restaurant  0.14
3            Restaurant  0.07
4    Mexican Restaurant  0.07


----Amraiwadi----
               venue  freq
0                ATM  0.67
1  Indian Restaurant  0.33
2      Moving Ta

4           Tea Room  0.17


----M D marg----
               venue  freq
0               Lake  0.25
1  Indian Restaurant  0.25
2        Golf Course  0.25
3           Platform  0.25
4  Martial Arts Dojo  0.00


----Makarba----
            venue  freq
0             ATM   0.5
1        Platform   0.5
2            Lake   0.0
3  Lingerie Store   0.0
4     Planetarium   0.0


----Makhiav----
               venue  freq
0    Fireworks Store   1.0
1                ATM   0.0
2  Recreation Center   0.0
3        Planetarium   0.0
4        Pizza Place   0.0


----Manek Chowk----
               venue  freq
0      Historic Site  0.17
1  Indian Restaurant  0.17
2            Theater  0.17
3    Bed & Breakfast  0.17
4           Tea Room  0.17


----Maninagar----
                  venue  freq
0  Fast Food Restaurant   0.5
1                Bakery   0.5
2         Moving Target   0.0
3              Platform   0.0
4           Planetarium   0.0


----Mankol----
               venue  freq
0    Fireworks Store  

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Locations_venues_sorted = pd.DataFrame(columns=columns)
Locations_venues_sorted['Location'] = ahmedabad_grouped['Location']

for ind in np.arange(ahmedabad_grouped.shape[0]):
    Locations_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ahmedabad_grouped.iloc[ind, :], num_top_venues)

Locations_venues_sorted.head(10)


,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahmedabad.,Snack Place,Indian Restaurant,Theater,Tea Room,Historic Site,Bed & Breakfast,Zoo,Electronics Store,Fast Food Restaurant,Fireworks Store
1,Ambaliara,Bank,Zoo,Golf Course,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
2,Ambareli,Bank,Zoo,Golf Course,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
3,Ambawadi,Indian Restaurant,Hotel,Restaurant,River,Food Court,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
4,Ambawadi Vistar,Indian Restaurant,BBQ Joint,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Café,Pizza Place,Restaurant,Coffee Shop,Mexican Restaurant,Tea Room
5,Amraiwadi,ATM,Indian Restaurant,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
6,Anandnagar,Indian Restaurant,Hotel,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
7,Andej,Lake,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
8,Asarwa Chakla,ATM,Tennis Stadium,Movie Theater,Indian Restaurant,Travel Lounge,Hotel Bar,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
9,Asarwa Ext south,Business Service,IT Services,Lake,Massage Studio,Bookstore,Spa,Zoo,Fast Food Restaurant,Fireworks Store,Fish Taverna


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [24]:
# set number of clusters
kclusters = 7

ahmedabad_grouped_clustering = ahmedabad_grouped.drop('Location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ahmedabad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
l=list(kmeans.labels_)


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
Locations_venues_sorted.insert(0,"Cluster_labels",l,allow_duplicates=False)
# merge ahmedabad_grouped with ahmedabad_data to add latitude/longitude for each neighborhood
ahmedabad_merged = pd.merge(df,Locations_venues_sorted,on='Location',how='inner')
ahmedabad_merged.head(10)

,Location,Name,Pincode,State,District,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahmedabad.,Ahmedabad.,380001,Gujarat,Ahmedabad,23.0198,72.5935,NaN,NaN,NaN,NaN,0,Snack Place,Indian Restaurant,Theater,Tea Room,Historic Site,Bed & Breakfast,Zoo,Electronics Store,Fast Food Restaurant,Fireworks Store
1,Ambaliara,Ambaliara,387810,Gujarat,Ahmedabad,22.7221,72.4361,NaN,NaN,NaN,NaN,2,Bank,Zoo,Golf Course,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
2,Ambareli,Ambareli,387810,Gujarat,Ahmedabad,22.7221,72.4361,NaN,NaN,NaN,NaN,2,Bank,Zoo,Golf Course,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
3,Ambawadi,Ambawadi,380006,Gujarat,Ahmedabad,23.0232,72.5699,NaN,NaN,NaN,NaN,4,Indian Restaurant,Hotel,Restaurant,River,Food Court,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
4,Ambawadi Vistar,Ambawadi Vistar,380015,Gujarat,Ahmedabad,23.012,72.513,NaN,NaN,NaN,NaN,0,Indian Restaurant,BBQ Joint,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Café,Pizza Place,Restaurant,Coffee Shop,Mexican Restaurant,Tea Room
5,Amraiwadi,Amraiwadi,380026,Gujarat,Ahmedabad,23.0025,72.6292,"2,041","6,011 / sqft","5,114.42 / sqft",1385.3%,1,ATM,Indian Restaurant,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
6,Anandnagar,Anandnagar,380007,Gujarat,Ahmedabad,23.0015,72.5432,NaN,NaN,NaN,NaN,4,Indian Restaurant,Hotel,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
7,Andej,Andej,382115,Gujarat,Ahmedabad,23.0707,72.4404,NaN,NaN,NaN,NaN,0,Lake,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
8,Asarwa Chakla,Asarwa Chakla,380016,Gujarat,Ahmedabad,23.0545,72.6023,NaN,NaN,NaN,NaN,0,ATM,Tennis Stadium,Movie Theater,Indian Restaurant,Travel Lounge,Hotel Bar,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
9,Asarwa Ext south,Asarwa Ext south,380024,Gujarat,Ahmedabad,23.0331,72.6184,NaN,NaN,NaN,NaN,0,Business Service,IT Services,Lake,Massage Studio,Bookstore,Spa,Zoo,Fast Food Restaurant,Fireworks Store,Fish Taverna


In [27]:
ahmedabad_merged.head(10)

,Location,Name,Pincode,State,District,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Amraiwadi,Amraiwadi,380026,Gujarat,Ahmedabad,23.0025,72.6292,"2,041","6,011 / sqft","5,114.42 / sqft",1385.3%,1,ATM,Indian Restaurant,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
10,Ashram Road,Ashram Road,380009,Gujarat,Ahmedabad,23.0243,72.5699,"2,881","7,234 / sqft","6,262.86 / sqft",-63.8%,4,Indian Restaurant,Hotel,Asian Restaurant,Restaurant,River,Food Truck,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
14,Bapunagar,Bapunagar,380024,Gujarat,Ahmedabad,23.0331,72.6184,"1,000","4,455 / sqft","3,747.6 / sqft",125.6%,0,Business Service,IT Services,Lake,Massage Studio,Bookstore,Spa,Zoo,Fast Food Restaurant,Fireworks Store,Fish Taverna
18,Bhat,Bhat,382210,Gujarat,Ahmedabad,22.9883,72.4731,"3,311","5,026 / sqft","4,761.62 / sqft",66.4%,4,Indian Restaurant,Restaurant,Sporting Goods Shop,IT Services,Food Court,Donut Shop,Ice Cream Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
20,Bodakdev,Bodakdev,380054,Gujarat,Ahmedabad,23.0453,72.5152,"2,653","22,500 / sqft","11,139.85 / sqft",-,0,Café,Indian Restaurant,Coffee Shop,Bistro,Shopping Mall,Sandwich Place,Mediterranean Restaurant,Ice Cream Shop,Pizza Place,Italian Restaurant
21,Bopal,Bopal,380058,Gujarat,Ahmedabad,23.0264,72.4764,393,"12,500 / sqft","8,15.91 / sqft",24.5%,0,Indian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Food Truck,Electronics Store,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
23,Chandlodia,Chandlodia,382481,Gujarat,Ahmedabad,23.0735,72.5605,"2,222","6,735 / sqft","5,359.19 / sqft",-75.1%,0,Tea Room,Zoo,Gift Shop,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
37,Ellisbridge,Ellisbridge,380006,Gujarat,Ahmedabad,23.0232,72.5699,"2,143","14,000 / sqft","9,320.82 / sqft",68.5%,4,Indian Restaurant,Hotel,Restaurant,River,Food Court,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
47,Ghuma,Ghuma,380058,Gujarat,Ahmedabad,23.0264,72.4764,"2,821","8,500 / sqft","6,311.42 / sqft",-11.9%,0,Indian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Food Truck,Electronics Store,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
49,Godhavi,Godhavi,382115,Gujarat,Ahmedabad,23.0707,72.4404,"1,500","3,030 / sqft","2,724.24 / sqft",-,0,Lake,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court


In [28]:
for col in ahmedabad_merged.columns: 
    print(col) 

Location
Name
Pincode
State
District
Lat
Long
Buy Rates 1
Buy Rates 3
Buy Rates Average
Trends
Cluster_labels
1st Most Common Venue
2nd Most Common Venue
3rd Most Common Venue
4th Most Common Venue
5th Most Common Venue
6th Most Common Venue
7th Most Common Venue
8th Most Common Venue
9th Most Common Venue
10th Most Common Venue


In [30]:
li=[]
for x in (0,len(ahmedabad_merged)):
    li.append(ahmedabad_merged["Cluster_labels"].iloc[:])

In [31]:
# create map
map_clusters = folium.Map(location=[latitude_ahmedabad, longitude_ahmedabad], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
a=0
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ahmedabad_merged['Lat'], ahmedabad_merged['Long'], ahmedabad_merged['Location'],li[a] ):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    a=a+1

map_clusters.choropleth(geo_data=country_geo,
            data=abadprices,
             columns=['Location', 'Buy Rates 1'],
             #key_on='feature.id',
             fill_color='BuGn' ,fill_opacity=0.3, line_opacity=0.7,
             )
       
map_clusters

/Users/ishaanbuch/opt/anaconda3/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Examine Clusters

#### Cluster 1

In [32]:
ahmedabad_merged.loc[ahmedabad_merged['Cluster_labels'] == 0, ahmedabad_merged.columns[[1] + list(range(5, ahmedabad_merged.shape[1]))]]

,Name,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Bapunagar,23.0331,72.6184,"1,000","4,455 / sqft","3,747.6 / sqft",125.6%,0,Business Service,IT Services,Lake,Massage Studio,Bookstore,Spa,Zoo,Fast Food Restaurant,Fireworks Store,Fish Taverna
20,Bodakdev,23.0453,72.5152,"2,653","22,500 / sqft","11,139.85 / sqft",-,0,Café,Indian Restaurant,Coffee Shop,Bistro,Shopping Mall,Sandwich Place,Mediterranean Restaurant,Ice Cream Shop,Pizza Place,Italian Restaurant
21,Bopal,23.0264,72.4764,393,"12,500 / sqft","8,15.91 / sqft",24.5%,0,Indian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Food Truck,Electronics Store,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
23,Chandlodia,23.0735,72.5605,"2,222","6,735 / sqft","5,359.19 / sqft",-75.1%,0,Tea Room,Zoo,Gift Shop,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
47,Ghuma,23.0264,72.4764,"2,821","8,500 / sqft","6,311.42 / sqft",-11.9%,0,Indian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Food Truck,Electronics Store,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
49,Godhavi,23.0707,72.4404,"1,500","3,030 / sqft","2,724.24 / sqft",-,0,Lake,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
52,Gota,23.0735,72.5605,"2,346","8,495 / sqft","5,955.76 / sqft",9.3%,0,Tea Room,Zoo,Gift Shop,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
63,Juhapura,22.9943,72.5295,"2,455","4,630 / sqft","4,38.98 / sqft",33%,0,Indian Restaurant,Bakery,Hotel,Flea Market,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
72,Khodiyar,23.1952,72.6146,"3,119","9,626 / sqft","6,179.17 / sqft",-,0,Indian Restaurant,Tea Room,Coffee Shop,Mexican Restaurant,Asian Restaurant,IT Services,Food Truck,Fast Food Restaurant,Fireworks Store,Fish Taverna
87,Maninagar,22.9926,72.609,"1,733","41,739 / sqft","17,446.84 / sqft",6.4%,0,Fast Food Restaurant,Bakery,Zoo,Golf Course,Electronics Store,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court


#### Cluster 2

In [33]:
ahmedabad_merged.loc[ahmedabad_merged['Cluster_labels'] == 1, ahmedabad_merged.columns[[1] + list(range(5, ahmedabad_merged.shape[1]))]]

,Name,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Amraiwadi,23.0025,72.6292,"2,041","6,011 / sqft","5,114.42 / sqft",1385.3%,1,ATM,Indian Restaurant,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
61,Jivraj Park,23.0037,72.5202,"4,188","4,960 / sqft","4,881.94 / sqft",5%,1,ATM,Platform,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
84,Makarba,23.0037,72.5202,"3,309","7,609 / sqft","6,349.49 / sqft",20.1%,1,ATM,Platform,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
119,Ranip,23.0825,72.5683,"2,844","10,345 / sqft","5,105.48 / sqft",17.4%,1,ATM,Sandwich Place,Food Truck,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
127,Saijpur Bogha,23.056,72.638,"1,112","3,105 / sqft","2,30.66 / sqft",-,1,ATM,Café,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
154,Vastral,23.0149,72.6361,"1,352","4,854 / sqft","3,616.13 / sqft",24.1%,1,ATM,Diner,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court,Food Truck
156,Vejalpur,23.0037,72.5202,"2,267","5,762 / sqft","5,164.4 / sqft",9.3%,1,ATM,Platform,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court


#### Cluster 3

In [34]:
ahmedabad_merged.loc[ahmedabad_merged['Cluster_labels'] == 2, ahmedabad_merged.columns[[1] + list(range(5, ahmedabad_merged.shape[1]))]]

,Name,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [35]:
ahmedabad_merged.loc[ahmedabad_merged['Cluster_labels'] == 3, ahmedabad_merged.columns[[1] + list(range(5, ahmedabad_merged.shape[1]))]]

,Name,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
129,Sanand,22.9937,72.382,611,"23,622 / sqft","3,956.32 / sqft",-28.1%,3,Fireworks Store,Zoo,Diner,Electronics Store,Fast Food Restaurant,Fish Taverna,Flea Market,Flower Shop,Food Court,Food Truck


#### Cluster 5

In [36]:
ahmedabad_merged.loc[ahmedabad_merged['Cluster_labels'] == 4, ahmedabad_merged.columns[[1] + list(range(5, ahmedabad_merged.shape[1]))]]

,Name,Lat,Long,Buy Rates 1,Buy Rates 3,Buy Rates Average,Trends,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Ashram Road,23.0243,72.5699,"2,881","7,234 / sqft","6,262.86 / sqft",-63.8%,4,Indian Restaurant,Hotel,Asian Restaurant,Restaurant,River,Food Truck,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
18,Bhat,22.9883,72.4731,"3,311","5,026 / sqft","4,761.62 / sqft",66.4%,4,Indian Restaurant,Restaurant,Sporting Goods Shop,IT Services,Food Court,Donut Shop,Ice Cream Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
37,Ellisbridge,23.0232,72.5699,"2,143","14,000 / sqft","9,320.82 / sqft",68.5%,4,Indian Restaurant,Hotel,Restaurant,River,Food Court,Donut Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
55,Hebatpur,22.2475,72.195,"4,701","7,353 / sqft","6,26.9 / sqft",50.4%,4,Indian Restaurant,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop,Food Court
103,Navrangpura,23.0243,72.5699,"3,203","11,746 / sqft","8,891.27 / sqft",-20.5%,4,Indian Restaurant,Hotel,Asian Restaurant,Restaurant,River,Food Truck,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna
110,Paldi,23.0015,72.5432,"3,516","21,667 / sqft","11,493.73 / sqft",14.4%,4,Indian Restaurant,Hotel,Zoo,Gift Shop,Electronics Store,Fast Food Restaurant,Fireworks Store,Fish Taverna,Flea Market,Flower Shop
130,Sanathal,22.9883,72.4731,"2,277","16,500 / sqft","7,316.09 / sqft",256.4%,4,Indian Restaurant,Restaurant,Sporting Goods Shop,IT Services,Food Court,Donut Shop,Ice Cream Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
134,Sarkhej,22.9883,72.4731,"1,167","4,773 / sqft","3,938.69 / sqft",20.8%,4,Indian Restaurant,Restaurant,Sporting Goods Shop,IT Services,Food Court,Donut Shop,Ice Cream Shop,Electronics Store,Fast Food Restaurant,Fireworks Store
